In [1]:
import datetime
import enum
import json
import logging
import os
import pickle
import random
import re
import shutil
import string
import time
import pandas as pd
from websocket import create_connection
import requests
import sys

logger = logging.getLogger(__name__)


class Interval(enum.Enum):
    in_1_minute = "1"
    in_3_minute = "3"
    in_5_minute = "5"
    in_15_minute = "15"
    in_30_minute = "30"
    in_45_minute = "45"
    in_1_hour = "1H"
    in_2_hour = "2H"
    in_3_hour = "3H"
    in_4_hour = "4H"
    in_daily = "1D"
    in_weekly = "1W"
    in_monthly = "1M"


class TvDatafeed:
    sign_in_url = 'https://www.tradingview.com/accounts/signin/'
    ws_headers = json.dumps({"Origin": "https://data.tradingview.com"})
    signin_headers = {'Referer': 'https://www.tradingview.com'}
    ws_timeout = 5

    def __init__(
        self,
        username: str = None,
        password: str = None,
    ) -> None:
        """Create TvDatafeed object

        Args:
            username (str, optional): tradingview username. Defaults to None.
            password (str, optional): tradingview password. Defaults to None.
        """

        self.ws_debug = False

        self.token = self.auth(username, password)

        if self.token is None:
            self.token = "unauthorized_user_token"
            logger.warning(
                "you are using nologin method, data you access may be limited"
            )

        self.ws = None
        self.session = self.generate_session()
        self.chart_session = self.generate_chart_session()

    def auth(self, username, password):

        if (username is None or password is None):
            token = None

        else:
            data = {"username": username,
                    "password": password,
                    "remember": "on"}
            try:
                response = requests.post(
                    url=self.sign_in_url, data=data, headers=self.signin_headers)
                token = response.json()['user']['auth_token']
            except Exception as e:
                logger.error('error while signin')
                token = None

        return token

    def create_connection(self):
        logging.debug("creating websocket connection")
        self.ws = create_connection(
            "wss://data.tradingview.com/socket.io/websocket", headers=self.ws_headers, timeout=self.ws_timeout
        )

    @staticmethod
    def filter_raw_message(text):
        try:
            found = re.search('"m":"(.+?)",', text).group(1)
            found2 = re.search('"p":(.+?"}"])}', text).group(1)

            return found, found2
        except AttributeError:
            logger.error("error in filter_raw_message")

    @staticmethod
    def generate_session():
        stringLength = 12
        letters = string.ascii_lowercase
        random_string = "".join(random.choice(letters)
                                for i in range(stringLength))
        return "qs_" + random_string

    @staticmethod
    def generate_chart_session():
        stringLength = 12
        letters = string.ascii_lowercase
        random_string = "".join(random.choice(letters)
                                for i in range(stringLength))
        return "cs_" + random_string

    @staticmethod
    def prepend_header(st):
        return "~m~" + str(len(st)) + "~m~" + st

    @staticmethod
    def construct_message(func, param_list):
        return json.dumps({"m": func, "p": param_list}, separators=(",", ":"))

    def create_message(self, func, paramList):
        return self.prepend_header(self.construct_message(func, paramList))

    def send_message(self, func, args):
        m = self.create_message(func, args)
        if self.ws_debug:
            print(m)
        self.ws.send(m)

    @staticmethod
    def create_df(raw_data, symbol):
        try:
            out = re.search('"s":\[(.+?)\}\]', raw_data).group(1)
            x = out.split(',{"')
            data = list()
            volume_data = True

            for xi in x:
                xi = re.split("\[|:|,|\]", xi)
                ts = datetime.datetime.fromtimestamp(float(xi[4]))

                row = [ts]

                for i in range(5, 10):

                    # skip converting volume data if does not exists
                    if not volume_data and i == 9:
                        row.append(0.0)
                        continue
                    try:
                        row.append(float(xi[i]))

                    except ValueError:
                        volume_data = False
                        row.append(0.0)
                        logger.debug('no volume data')

                data.append(row)

            data = pd.DataFrame(
                data, columns=["datetime", "open",
                               "high", "low", "close", "volume"]
            ).set_index("datetime")
            data.insert(0, "symbol", value=symbol)
            return data
        except AttributeError:
            logger.error("no data, please check the exchange and symbol")

    @staticmethod
    def format_symbol(symbol, exchange, contract: int = None):

        if ":" in symbol:
            pass
        elif contract is None:
            symbol = f"{exchange}:{symbol}"

        elif isinstance(contract, int):
            symbol = f"{exchange}:{symbol}{contract}!"

        else:
            raise ValueError("not a valid contract")

        return symbol

    def get_hist(
        self,
        symbol: str,
        exchange: str = "NSE",
        interval: Interval = Interval.in_daily,
        n_bars: int = 10,
        fut_contract: int = None,
        extended_session: bool = False,
    ) -> pd.DataFrame:
        """get historical data

        Args:
            symbol (str): symbol name
            exchange (str, optional): exchange, not required if symbol is in format EXCHANGE:SYMBOL. Defaults to None.
            interval (str, optional): chart interval. Defaults to 'D'.
            n_bars (int, optional): no of bars to download, max 5000. Defaults to 10.
            fut_contract (int, optional): None for cash, 1 for continuous current contract in front, 2 for continuous next contract in front . Defaults to None.
            extended_session (bool, optional): regular session if False, extended session if True, Defaults to False.

        Returns:
            pd.Dataframe: dataframe with sohlcv as columns
        """
        symbol = self.format_symbol(
            symbol=symbol, exchange=exchange, contract=fut_contract
        )

        interval = interval.value

        self.create_connection()

        self.send_message("set_auth_token", [self.token])
        self.send_message("chart_create_session", [self.chart_session, ""])
        self.send_message("quote_create_session", [self.session])
        self.send_message(
            "quote_set_fields",
            [
                self.session,
                "ch",
                "chp",
                "current_session",
                "description",
                "local_description",
                "language",
                "exchange",
                "fractional",
                "is_tradable",
                "lp",
                "lp_time",
                "minmov",
                "minmove2",
                "original_name",
                "pricescale",
                "pro_name",
                "short_name",
                "type",
                "update_mode",
                "volume",
                "currency_code",
                "rchp",
                "rtc",
            ],
        )

        print(self.session)

        self.send_message(
            "quote_add_symbols", [self.session, symbol,
                                  {"flags": ["force_permission"]}]
        )
        self.send_message("quote_fast_symbols", [self.session, symbol])

        self.send_message(
            "resolve_symbol",
            [
                self.chart_session,
                "symbol_1",
                '={"symbol":"'
                + symbol
                + '","adjustment":"splits","session":'
                + ('"regular"' if not extended_session else '"extended"')
                + "}",
            ],
        )
        self.send_message(
            "create_series",
            [self.chart_session, "s1", "s1", "symbol_1", interval, n_bars],
        )
        self.send_message("switch_timezone", [
                            self.chart_session, "exchange"])

        raw_data = ""

        logger.debug(f"getting data for {symbol}...")
        while True:
            try:
                result = self.ws.recv()
                raw_data = raw_data + result + "\n"
            except Exception as e:
                logger.error(e)
                break

            if "series_completed" in result:
                break

        return self.create_df(raw_data, symbol)


if __name__ == "__main__":
    logging.basicConfig(level=logging.DEBUG)
    tv = TvDatafeed(
    )
    print(tv.get_hist("CRUDEOIL", "MCX", fut_contract=1))
    # print(tv.get_hist("NIFTY", "NSE", fut_contract=1))
    # print(
    #     tv.get_hist(
    #         "EICHERMOT",
    #         "NSE",
    #         interval=Interval.in_1_hour,
    #         n_bars=500,
    #         extended_session=False,
    #     )
    # )


DEBUG:root:creating websocket connection
DEBUG:__main__:getting data for MCX:CRUDEOIL1!...


qs_owsbyvmwwgel


ERROR:__main__:The read operation timed out
ERROR:__main__:no data, please check the exchange and symbol


None


In [2]:

username = 'andrzolide@gmail.com'
password = 'Zadymka123'
# initialize tradingview

tv = TvDatafeed(username=username,password=password)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.tradingview.com:443
DEBUG:urllib3.connectionpool:https://www.tradingview.com:443 "POST /accounts/signin/ HTTP/1.1" 200 None


In [3]:
tv.get_hist('AAPL','NASDAQ',)

DEBUG:root:creating websocket connection
DEBUG:__main__:getting data for NASDAQ:AAPL...


qs_apmsvpzimsfm


,symbol,open,high,low,close,volume
datetime,,,,,,
2022-09-13 15:30:00,NASDAQ:AAPL,159.900,160.5400,153.3700,153.84,122656614.0
2022-09-14 15:30:00,NASDAQ:AAPL,154.785,157.1000,153.6106,155.31,87965409.0
2022-09-15 15:30:00,NASDAQ:AAPL,154.650,155.2400,151.3800,152.37,90481110.0
2022-09-16 15:30:00,NASDAQ:AAPL,151.210,151.3500,148.3700,150.70,162278841.0
2022-09-19 15:30:00,NASDAQ:AAPL,149.310,154.5600,149.1000,154.48,81474246.0
2022-09-20 15:30:00,NASDAQ:AAPL,153.400,158.0800,153.0800,156.90,107689796.0
2022-09-21 15:30:00,NASDAQ:AAPL,157.340,158.7400,153.6000,153.72,101696790.0
2022-09-22 15:30:00,NASDAQ:AAPL,152.380,154.4700,150.9100,152.74,86652542.0
2022-09-23 15:30:00,NASDAQ:AAPL,151.190,151.4700,148.5600,150.43,96029909.0


In [37]:

symbol = 'AAPL'
exchange: str = "NASDAQ"
interval: Interval = Interval.in_5_minute
n_bars: int = 10
fut_contract: int = None
extended_session: bool = False

"""get historical data

Args:
    symbol (str): symbol name
    exchange (str, optional): exchange, not required if symbol is in format EXCHANGE:SYMBOL. Defaults to None.
    interval (str, optional): chart interval. Defaults to 'D'.
    n_bars (int, optional): no of bars to download, max 5000. Defaults to 10.
    fut_contract (int, optional): None for cash, 1 for continuous current contract in front, 2 for continuous next contract in front . Defaults to None.
    extended_session (bool, optional): regular session if False, extended session if True, Defaults to False.

Returns:
    pd.Dataframe: dataframe with sohlcv as columns
"""
symbol = tv.format_symbol(
    symbol=symbol, exchange=exchange
)

interval = interval.value

tv.create_connection()

tv.send_message("set_auth_token", [tv.token])
tv.send_message("chart_create_session", [tv.chart_session, ""])
tv.send_message("quote_create_session", [tv.session])
tv.send_message(
    "quote_set_fields",
    [
        tv.session,
        "ch",
        "chp",
        "current_session",
        "description",
        "local_description",
        "language",
        "exchange",
        "fractional",
        "is_tradable",
        "lp",
        "lp_time",
        "minmov",
        "minmove2",
        "original_name",
        "pricescale",
        "pro_name",
        "short_name",
        "type",
        "update_mode",
        "volume",
        "currency_code",
        "rchp",
        "rtc",
    ],
)

print(tv.session)

tv.send_message(
    "quote_add_symbols", [tv.session, symbol,
                            {"flags": ["force_permission"]}]
)
tv.send_message("quote_fast_symbols", [tv.session, symbol])

tv.send_message(
    "resolve_symbol",
    [
        tv.chart_session,
        "symbol_1",
        '={"symbol":"'
        + symbol
        + '","adjustment":"splits","session":'
        + ('"regular"' if not extended_session else '"extended"')
        + "}",
    ],
)
tv.send_message(
    "create_series",
    [tv.chart_session, "sds_1", "s1", "symbol_1", interval, n_bars],
)
tv.send_message("switch_timezone", [
                    tv.chart_session, "exchange"])

raw_data = ""

logger.debug(f"getting data for {symbol}...")
while True:
    try:
        result = tv.ws.recv()
        # print(result)
        raw_data = raw_data + result + "\n"
    except Exception as e:
        logger.error(e)
        break

    if "series_completed" in result:
        print('completed 1')
        break

df = tv.create_df(raw_data, symbol)

list_dfs = []
list_dfs.append(df)
import time
for i in range(5):
    tv.send_message(
        "request_more_data",
        [tv.chart_session, "sds_1", 1000],
    )
    # result = tv.ws.recv()
    raw_data2 = ""
    while True:
        try:
            result = tv.ws.recv()
            # print(result)
            raw_data2 = raw_data2 + result + "\n"
        except Exception as e:
            logger.error(e)
            break

        if "series_completed" in result:
            print('completed 2')

            break
        
        if result == '':
            break
    time.sleep(1)

    df2 = tv.create_df(raw_data2, symbol)
    list_dfs.append(df2)

DEBUG:root:creating websocket connection
DEBUG:__main__:getting data for NASDAQ:AAPL...


qs_apmsvpzimsfm
completed 1
completed 2
completed 2
completed 2
completed 2
completed 2


In [38]:
list_dfs

[                          symbol     open    high     low   close    volume
 datetime                                                                   
 2022-09-26 21:10:00  NASDAQ:AAPL  151.650  151.77  151.48  151.53   44288.0
 2022-09-26 21:15:00  NASDAQ:AAPL  151.540  151.86  151.19  151.67   51546.0
 2022-09-26 21:20:00  NASDAQ:AAPL  151.680  151.83  151.42  151.58   32970.0
 2022-09-26 21:25:00  NASDAQ:AAPL  151.580  151.74  151.42  151.66   26089.0
 2022-09-26 21:30:00  NASDAQ:AAPL  151.680  151.68  151.31  151.41   38237.0
 2022-09-26 21:35:00  NASDAQ:AAPL  151.430  151.75  151.26  151.33   63726.0
 2022-09-26 21:40:00  NASDAQ:AAPL  151.335  151.67  150.95  151.22   61597.0
 2022-09-26 21:45:00  NASDAQ:AAPL  151.230  151.48  151.14  151.36   80008.0
 2022-09-26 21:50:00  NASDAQ:AAPL  151.360  152.00  150.89  151.57  191493.0
 2022-09-26 21:55:00  NASDAQ:AAPL  151.570  151.57  150.56  150.76  332194.0,
                           symbol     open    high      low   close   volum